### 1. Scrape of 2021 Brazilian soccer championship classification

In [2]:
import pandas as pd
import re
pd.set_option("display.max_rows", 20, "display.max_columns", 20)

page = pd.read_html('https://www.uol.com.br/esporte/futebol/campeonatos/brasileirao')
df = pd.concat([pd.DataFrame(page[0]), pd.DataFrame(page[1])], axis=1)
df.rename(columns = {'classificação':'Ranking'}, inplace = True)
df[['Ranking', 'Team']] = df['Ranking'].str.split('°', 1, expand=True)
df['Team'] = df['Team'].apply(lambda x:re.split(r'[A-Z]{3}$', x)[0].strip())
cols = df.columns.tolist()
cols = [cols[0], cols[-1]] + cols[1:-1]
df = df[cols]
df.to_excel('ranking_brasileirao.xlsx')
df

,Ranking,Team,PG,J,V,E,D,GC,GP,SG,%
0,1,Atlético-MG,71,32,22,5,5,22,51,29,74
1,2,Flamengo,63,32,19,6,7,28,62,34,66
2,3,Palmeiras,58,33,18,4,11,39,52,13,59
3,4,Red Bull Bragantino,52,34,13,13,8,40,50,10,51
4,5,Corinthians,50,33,13,11,9,32,35,3,51
5,6,Fortaleza,49,33,14,7,12,41,40,-1,49
6,7,Internacional,47,33,12,11,10,35,41,6,47
7,8,Fluminense,45,33,12,9,12,34,31,-3,45
8,9,América-MG,45,33,11,12,10,34,35,1,45
9,10,Ceará,45,33,10,15,8,33,35,2,45


### 2. Scrape of all 380 matches of 2021 Brazilian soccer championship

In [3]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

url = 'https://www.uol.com.br/esporte/futebol/campeonatos/brasileirao'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(req).read()
page = soup(html, 'html.parser')
containers = page.findAll('ul', class_ = 'rounds-content no-change')
tag = containers[0].text

games = re.split('Sáb|Dom|Seg|Ter|Qua|Qui|Sex|pós-jogo', tag)
games = [g.replace(',', ' ').strip() for g in games if g.strip() != '']
for i in range(len(games)):
    res = re.search(r'^\d\d/\d\d\s-\s\d\dh\d*', games[i])
    if res == None: 
        games[i] = re.search(r'^\d\d/\d\d\s-\s\d\dh\d*', games[i+1])[0] + ' - ' + games[i]
games = [re.sub('      |\t|   ', ' - ', g).strip() for g in games if len(g.split('-')) > 2]
games = [re.split(r' -  - | - |\(\)', g) for g in games]
df = pd.DataFrame(games, columns = ['Date', 'Time', 'Place', 'Team1', 'Team2', 'delete']).iloc[:,:-1]
df[['Score1', 'Score2']] = ''
for i in range(len(df)):
    txt1 = df.loc[i,'Team1'].strip()[3:]
    score1 = re.findall(r'\d+|-$', txt1)[0]
    if score1== '-': score1=''
    txt2 = df.loc[i,'Team2'].strip()[:-3]
    score2 = re.findall(r'^\d+|^-', txt2)[0]
    if score2== '-': score2=''
    df.loc[i, 'Team1'] = re.split(r'\d+|-$', txt1)[0]
    df.loc[i, 'Team2'] = re.split(r'^\d+|^-', txt2)[1]
    df.loc[i, 'Score1'] = score1
    df.loc[i, 'Score2'] = score2
df.to_excel('matches_brasileirao.xlsx')
df

,Date,Time,Place,Team1,Team2,Score1,Score2
0,29/05,19h,Arena Pantanal,Cuiabá,Juventude,2,2
1,29/05,20h,Pituaçu,Bahia,Santos,3,0
2,29/05,21h,Morumbi,São Paulo,Fluminense,0,0
3,30/05,11h,Mineirão,Atlético-MG,Fortaleza,1,2
4,30/05,16h,Maracanã,Flamengo,Palmeiras,1,0
...,...,...,...,...,...,...,...
375,09/12,21h30,Castelão (CE),Fortaleza,Bahia,,
376,09/12,21h30,Arena Pernambuco,Sport,Athletico,,
377,09/12,21h30,Nabi Abi Chedid,Red Bull Bragantino,Internacional,,
378,09/12,21h30,Alfredo Jaconi,Juventude,Corinthians,,
